In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessing/train.csv
/kaggle/input/preprocessing/test.csv


In [2]:
#importing the training data
imdb_data=pd.read_csv('/kaggle/input/preprocessing/train.csv')
imdb_data_test=pd.read_csv('/kaggle/input/preprocessing/test.csv')
print(imdb_data.shape)
imdb_data.head(10)

(30000, 2)


,review,sentiment
0,SAPS AT SEA <br /><br />Aspect ratio: 1.37:1<b...,negative
1,"If you want mindless action, hot chicks and a ...",positive
2,"""The Woman in Black"" is easily one of the cree...",positive
3,I can barely find the words to describe how mu...,negative
4,What's in here ?! Let me tell you. It's the pr...,negative
5,"This is the story of a maniac cop who, for som...",negative
6,Before I continue forth with the new millenniu...,positive
7,"When Rodney Dangerfield is on a roll, he's hil...",negative
8,Prom Night is shot with the artistic eye someo...,negative
9,"""Destroy All Planets"" winds up settling for 'd...",negative


In [3]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch
import numpy as np
import time
from sklearn.metrics import accuracy_score

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

# Set pad token id to eos token id
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# Ensure the model is on CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def tokenize_data(reviews, labels, max_length):
    input_ids = []
    attention_masks = []

    for review in reviews:
        encoded_data = tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids.append(encoded_data['input_ids'])
        attention_masks.append(encoded_data['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

# Convert labels to 0 and 1
train_labels = [1 if label == 'positive' else 0 for label in imdb_data['sentiment']]
test_labels = [1 if label == 'positive' else 0 for label in imdb_data_test['sentiment']]

# Tokenize the dataset
train_input_ids, train_attention_masks, train_labels = tokenize_data(imdb_data['review'], train_labels, max_length=256)
test_input_ids, test_attention_masks, test_labels = tokenize_data(imdb_data_test['review'], test_labels, max_length=256)

# Create DataLoaders
batch_size = 6  # GPT-2 is large; you may need to use a small batch size
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=3e-5, eps=1e-8)
epochs = 5
total_steps = len(train_dataloader) * epochs
warmup_steps = int(total_steps * 0.15)  # 10% of total steps as warmup
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Helper function for accuracy calculation
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Training loop
total_training_time_start = time.time()

for epoch_i in range(epochs):
    epoch_start_time = time.time()
    
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    epoch_time = time.time() - epoch_start_time
    print(f"Epoch {epoch_i + 1}/{epochs} completed. Loss: {total_loss / len(train_dataloader)}. Time: {epoch_time:.2f} sec.")

# Evaluation
evaluation_start_time = time.time()
model.eval()
eval_accuracy = 0
nb_eval_steps = 0
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_input_mask)
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

evaluation_time = time.time() - evaluation_start_time
total_training_time = time.time() - total_training_time_start
print(f"Accuracy on test set: {eval_accuracy / nb_eval_steps}. Evaluation time: {evaluation_time:.2f} sec.")
print(f"Total training and evaluation time: {total_training_time:.2f} sec.")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 completed. Loss: 0.38760852175116595. Time: 1657.43 sec.
Epoch 2/5 completed. Loss: 0.2858512156621277. Time: 1658.78 sec.
Epoch 3/5 completed. Loss: 0.20278233495205641. Time: 1658.44 sec.
Epoch 4/5 completed. Loss: 0.1383790847783399. Time: 1655.97 sec.
Epoch 5/5 completed. Loss: 0.09269599127912442. Time: 1659.08 sec.
Accuracy on test set: 0.9266146770645942. Evaluation time: 337.05 sec.
Total training and evaluation time: 8626.75 sec.
